In [1]:
import os
import cv2
import numpy as np
from PIL import Image
from tqdm import tqdm

# Define the root directory containing pest images
root_dir = "./"

# Target size
IMG_SIZE = (224, 224)

def smart_crop_and_resize(image_path, save_path):
    """Smartly crops around the pest and resizes the image to (224, 224) while keeping color."""
    try:
        # Read the image in color
        img = cv2.imread(image_path, cv2.IMREAD_COLOR)

        if img is None:
            print(f"❌ Skipping {image_path} (could not read)")
            return
        
        # Convert to grayscale only for processing (not saving)
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

        # Apply thresholding to detect the main object
        _, thresh = cv2.threshold(gray, 30, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

        # Find contours
        contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        if contours:
            # Get the largest contour (assuming it's the pest)
            c = max(contours, key=cv2.contourArea)
            x, y, w, h = cv2.boundingRect(c)
            
            # Crop to detected region (expand slightly for better framing)
            padding = 10
            x = max(0, x - padding)
            y = max(0, y - padding)
            w = min(img.shape[1] - x, w + 2 * padding)
            h = min(img.shape[0] - y, h + 2 * padding)
            
            cropped_img = img[y:y+h, x:x+w]  # Keep original color
        else:
            cropped_img = img  # If no object found, use the full image

        # Convert to RGB format for EfficientNetV2 compatibility
        pil_img = Image.fromarray(cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB))

        # Resize using LANCZOS for high quality
        resized_img = pil_img.resize(IMG_SIZE, Image.LANCZOS)

        # Save the processed image
        resized_img.save(save_path, "JPEG", quality=95)

    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")

# Process all images in dataset
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    
    if os.path.isdir(folder_path):  # Ensure it's a folder
        print(f"Processing folder: {folder}")
        
        for filename in tqdm(os.listdir(folder_path), desc=f"Resizing in {folder}"):
            if filename.lower().endswith(".jpg"):
                img_path = os.path.join(folder_path, filename)
                smart_crop_and_resize(img_path, img_path)  # Overwrite the image

print("\n✅ All images resized and centered successfully while keeping color!")


Processing folder: .dist


Resizing in .dist: 0it [00:00, ?it/s]


Processing folder: .qodo


Resizing in .qodo: 100%|██████████| 1/1 [00:00<?, ?it/s]


Processing folder: aphids


...

Processing folder: Thrips tabaci


Resizing in Thrips tabaci: 100%|██████████| 500/500 [00:08<00:00, 60.48it/s]


Processing folder: weevil


Resizing in weevil: 100%|██████████| 500/500 [00:04<00:00, 115.96it/s]


✅ All images resized and centered successfully while keeping color!


In [2]:
import os
import cv2
import numpy as np
import hashlib
from tqdm import tqdm


# Dictionary to store image hashes
image_hashes = {}

def compute_image_hash(image_path):
    """Computes a hash for an image using OpenCV and hashlib."""
    try:
        img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
        img = cv2.resize(img, (64, 64))  # Resize for hashing consistency
        img_hash = hashlib.md5(img.tobytes()).hexdigest()  # Compute hash
        return img_hash
    except Exception as e:
        print(f"❌ Error processing {image_path}: {e}")
        return None

# Scan and remove duplicate images
for folder in os.listdir(root_dir):
    folder_path = os.path.join(root_dir, folder)
    
    if os.path.isdir(folder_path):  # Ensure it's a folder
        print(f"Checking duplicates in: {folder}")

        for filename in tqdm(os.listdir(folder_path), desc=f"Scanning {folder}"):
            if filename.lower().endswith(".jpg"):
                img_path = os.path.join(folder_path, filename)
                img_hash = compute_image_hash(img_path)

                if img_hash:
                    if img_hash in image_hashes:
                        print(f"🗑️ Removing duplicate: {img_path}")
                        os.remove(img_path)  # Delete duplicate
                    else:
                        image_hashes[img_hash] = img_path  # Store unique image hash

print("\n✅ Duplicate images removed successfully!")


Checking duplicates in: .dist


Scanning .dist: 0it [00:00, ?it/s]


Checking duplicates in: .qodo


Scanning .qodo: 100%|██████████| 1/1 [00:00<?, ?it/s]


Checking duplicates in: aphids


...

Checking duplicates in: weevil


Scanning weevil:  25%|██▌       | 127/500 [00:00<00:00, 1264.03it/s]

🗑️ Removing duplicate: ./weevil\Weevil (111).jpg
🗑️ Removing duplicate: ./weevil\Weevil (179).jpg
🗑️ Removing duplicate: ./weevil\Weevil (199).jpg


Scanning weevil: 100%|██████████| 500/500 [00:00<00:00, 1319.23it/s]


✅ Duplicate images removed successfully!


In [14]:
import os
import numpy as np
import pandas as pd
from glob import glob

VALID_EXTENSIONS = {".jpg"}  # Valid image formats

# Dictionary to store image counts per class
image_counts = {}

# Loop through each class folder
for pest_class in os.listdir(root_dir):
    class_dir = os.path.join(root_dir, pest_class)
    if os.path.isdir(class_dir):  # Ensure it's a directory
        num_images = sum(
            len(glob(os.path.join(class_dir, f"*{ext}"))) for ext in VALID_EXTENSIONS
        )
        image_counts[pest_class] = num_images

# Convert to DataFrame for better visualization
df_counts = pd.DataFrame(image_counts.items(), columns=["Class", "Image Count"])

# Display total images in the dataset
total_images = df_counts["Image Count"].sum()
print(f"📌 Total images in dataset: {total_images}")
df_counts

📌 Total images in dataset: 5471


,Class,Image Count
0,.dist,0
1,.qodo,0
2,aphids,389
3,aphis craccivora,411
4,beet armyworm,455
5,catterpillar,474
6,Empoasca fabae,463
7,grasshopper,462
8,Helicoverpa armigera,310
9,mites,364


In [15]:
%pip install Pillow tqdm

Note: you may need to restart the kernel to use updated packages.


In [16]:
import os
import random
from PIL import Image, ImageEnhance, ImageOps
from tqdm import tqdm

# Settings
target_classes = {
    "Helicoverpa armigera": 450,
    "Thrips tabaci": 450,
    "mites": 450
}
base_dir = "./" 

def augment_image(image):
    """Apply random augmentation."""
    operations = [
        lambda x: x.rotate(random.randint(-25, 25)),
        lambda x: ImageOps.mirror(x),
        lambda x: ImageOps.flip(x),
        lambda x: ImageEnhance.Brightness(x).enhance(random.uniform(0.7, 1.3)),
        lambda x: ImageEnhance.Contrast(x).enhance(random.uniform(0.7, 1.3)),
        lambda x: ImageEnhance.Color(x).enhance(random.uniform(0.7, 1.3)),
    ]
    op = random.choice(operations)
    return op(image)

for class_name, target_count in target_classes.items():
    class_dir = os.path.join(base_dir, class_name)
    images = [f for f in os.listdir(class_dir) if f.lower().endswith(('.jpg', '.png', '.jpeg'))]
    current_count = len(images)
    print(f"{class_name}: {current_count} ➡ {target_count}")

    if current_count >= target_count:
        continue

    augment_needed = target_count - current_count
    for i in tqdm(range(augment_needed), desc=f"Augmenting {class_name}"):
        img_name = random.choice(images)
        img_path = os.path.join(class_dir, img_name)

        try:
            img = Image.open(img_path)
            img = augment_image(img)
            new_img_name = f"aug_{i}_{img_name}"
            img.save(os.path.join(class_dir, new_img_name))
        except Exception as e:
            print(f"❌ Error processing {img_name}: {e}")


Helicoverpa armigera: 310 ➡ 450


Augmenting Helicoverpa armigera: 100%|██████████| 140/140 [00:01<00:00, 113.31it/s]


Thrips tabaci: 357 ➡ 450


Augmenting Thrips tabaci: 100%|██████████| 93/93 [00:00<00:00, 136.71it/s]


mites: 364 ➡ 450


Augmenting mites: 100%|██████████| 86/86 [00:00<00:00, 133.39it/s]


In [17]:
import os
import numpy as np
import pandas as pd
from glob import glob

VALID_EXTENSIONS = {".jpg"}  # Valid image formats

# Dictionary to store image counts per class
image_counts = {}

# Loop through each class folder
for pest_class in os.listdir(root_dir):
    class_dir = os.path.join(root_dir, pest_class)
    if os.path.isdir(class_dir):  # Ensure it's a directory
        num_images = sum(
            len(glob(os.path.join(class_dir, f"*{ext}"))) for ext in VALID_EXTENSIONS
        )
        image_counts[pest_class] = num_images

# Convert to DataFrame for better visualization
df_counts = pd.DataFrame(image_counts.items(), columns=["Class", "Image Count"])

# Display total images in the dataset
total_images = df_counts["Image Count"].sum()
print(f"📌 Total images in dataset: {total_images}")
df_counts

📌 Total images in dataset: 5790


,Class,Image Count
0,.dist,0
1,.qodo,0
2,aphids,389
3,aphis craccivora,411
4,beet armyworm,455
5,catterpillar,474
6,Empoasca fabae,463
7,grasshopper,462
8,Helicoverpa armigera,450
9,mites,450
